# Model 2 for Leaf Image Classification Project

### Load Packages and Images & Create Model

In [ ]:
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D, BatchNormalization
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Activation, Flatten, Dense
import tensorflow as tf

HEIGHT = 150
WIDTH = 150
DEPTH = 3
VAL_SPLIT = 0.2
BS = 15
LEARNING_RATE = 0.001
EPOCHS = 7
DIR = r"C:\Users\Taja\Documents\MSDAIS Spring 2021\CIS 5367 Machine Learning\Spark Projects\data_all"

CATEGORIES = ['poison_ivy', 'raspberry', 'tomato', 'potato', 'bell pepper']
N_CLASSES = len(CATEGORIES)

features, labels = load_data()

#split data
(x_interim, x_test, y_interim, y_test) = train_test_split(features,labels, test_size =0.2)

(x_train, x_val, y_train, y_val) = train_test_split(x_interim, y_interim, test_size =0.2)



#---------------- model building---------------
def model_building():
    checkpoint = ModelCheckpoint('weights.h5', monitor = 'val_loss', save_best_only=True)
    early_stopping_monitor = EarlyStopping(patience=3)
    callbacks_list = [checkpoint, early_stopping_monitor]
    
    #-------------------OPTIMIZE AND REDUCE LOSS-----------------
    print("Optimizing and training network")
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
       
    model = Sequential()
    inputShape = (HEIGHT, WIDTH, DEPTH)
    chanDim = -1
    if K.image_data_format() == "channels_first":
        inputShape = (DEPTH, HEIGHT, WIDTH)
        chanDim = 1
    model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    
    model.add(Dense(N_CLASSES))
    model.add(Activation("softmax"))
             
    #-----------------Model Summary-----------------
    #model.summary()
    
    #compile model
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    
    #image augmentation

    aug = ImageDataGenerator(
        rotation_range=25, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, 
        zoom_range=0.2,horizontal_flip=True, 
        fill_mode="nearest")
    
    history = model.fit_generator(
        aug.flow(x_train, y_train, batch_size=BS),
        validation_data=(x_val, y_val),
        steps_per_epoch=len(x_train) // BS,
        epochs=EPOCHS, 
        verbose=1, callbacks = callbacks_list
        )
     
    return model,history

def eval_model(model,history):
    import matplotlib.pyplot as plt
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    #Train and validation accuracy
    plt.plot(epochs, acc, 'b', label='Training accurarcy')
    plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
    plt.title('Training and Validation accurarcy')
    plt.legend()
    
    plt.figure()
    #Train and validation loss
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.legend()
    plt.show()
    
    print("Calculating model accuracy")
    scores = model.evaluate(x_test, y_test)
    print(f"Test Accuracy: {scores[1]*100}%")

def save_model(model):
    model.save(DIR + r"\project_model.h5")

def load_model():
    import keras
    model = keras.models.load_model(DIR + r"\project_model.h5")
    return model

### Run Model

In [ ]:
def main():
    model, history = model_building()
    eval_model(model, history)

In [ ]:
main()